# Libs

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, GlobalMaxPooling1D, LayerNormalization, MultiHeadAttention, Add, Flatten, Concatenate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from utils.futurai_ppd import drop_transitorio_desligado

import warnings
import os
warnings.filterwarnings('ignore')

# Import dataset

In [ ]:
base_name = 'Depurador 762-28-006'
timestamp = "Timestamp"

df_dataset = pd.read_csv('data/' + base_name + '.csv', sep=";", decimal=".", encoding="utf-8-sig")
df_dataset[timestamp] = pd.to_datetime(df_dataset[timestamp], format="%Y-%m-%d %H:%M:%S")
df_dataset.drop(columns=["762H0336.PV", "762H0342.PV", "762N0015.SP", "762P0013.SP"], inplace=True)
df_dataset.dropna(inplace=True)
print(f"Dataset shape: {df_dataset.shape}")
df_dataset.head()

## Remove periods Off

In [ ]:
pre_process = []
pp_var_ref_desligado = "762-28-006.CR"
pp_valor_ref_desligado = 5
pp_tempo_ref_desligado = 0
pp_pre_corte_transitorio = 0
pp_pos_corte_transitorio = 0
pre_process.append(  
{
   "after_cut": pp_pos_corte_transitorio,
   "interval_off": pp_tempo_ref_desligado,
   "limit_off": pp_valor_ref_desligado,
   "pre_cut": pp_pre_corte_transitorio,
   "variable_off": pp_var_ref_desligado
  })

for pro in pre_process:
    df_dataset,_,_ = drop_transitorio_desligado(df_dataset,pro["variable_off"],pro["limit_off"],pro["interval_off"],timestamp,pre_corte=pro["pre_cut"],pos_corte=pro["after_cut"])
print(f"Dataset shape: {df_dataset.shape}")
df_dataset.head()

## Create label for anomaly

In [ ]:
periodos_de_Falhas = [
    (pd.Timestamp('2024-05-03 11:00:00'), pd.Timestamp('2024-05-03 11:35:00')),
    (pd.Timestamp('2024-06-25 17:20:00'), pd.Timestamp('2024-08-02 14:00:00')),
    (pd.Timestamp('2024-10-19 10:40:00'), pd.Timestamp('2024-10-19 10:50:00')),
    (pd.Timestamp('2024-10-19 10:40:00'), pd.Timestamp('2024-10-19 10:50:00')),
    (pd.Timestamp('2024-10-21 12:00:00'), pd.Timestamp('2024-10-22 00:35:00')),
    (pd.Timestamp('2024-10-24 03:10:00'), pd.Timestamp('2024-10-27 00:00:00')),
    (pd.Timestamp('2024-11-14 06:40:00'), pd.Timestamp('2024-11-14 19:45:00')),
    (pd.Timestamp('2024-11-25 21:45:00'), pd.Timestamp('2024-11-25 22:03:00')),
    (pd.Timestamp('2024-11-27 15:00:00'), pd.Timestamp('2024-11-27 15:07:00')),
    (pd.Timestamp('2024-11-27 16:04:00'), pd.Timestamp('2024-11-26 16:11:00')),
    (pd.Timestamp('2024-11-30 13:30:00'), pd.Timestamp('2024-11-30 15:52:00')),
    (pd.Timestamp('2024-12-09 20:30:00'), pd.Timestamp('2024-12-11 07:00:00')),
    (pd.Timestamp('2025-03-05 15:17:00'), pd.Timestamp('2025-03-05 15:24:00')),
    (pd.Timestamp('2025-03-15 18:30:00'), pd.Timestamp('2025-03-15 19:15:00')),
    (pd.Timestamp('2025-03-18 11:40:00'), pd.Timestamp('2025-03-18 20:00:00')),
    (pd.Timestamp('2025-06-16 15:20:00'), pd.Timestamp('2025-06-17 07:38:00')),
]

# Inicializa a nova coluna com valor 0
df_dataset['Falhas'] = 0

# Para cada período de Falhas, marca os índices onde o timestamp está dentro desse período
for inicio, fim in periodos_de_Falhas:
    df_dataset.loc[(df_dataset[timestamp] >= inicio) & (df_dataset[timestamp] <= fim), 'Falhas'] = 1
    
df_dataset

## Import TAGs and descriptions

In [ ]:
df_subsistema = pd.read_csv('data/'+ base_name + '_subsistema.csv', sep=";", decimal=".", encoding="utf-8-sig")
df_subsistema

## Plot variables

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_dataset['Timestamp'], 
    y=df_dataset["762P0034.PV"], 
    mode='lines', 
    name='762P0034.PV',
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=df_dataset['Timestamp'], 
    y=df_dataset["Falhas"], 
    mode='lines', 
    name='Falhas',
    line=dict(color='red')
))

fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
)
fig.show()